In [0]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from os import listdir
from os.path import isfile, join

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [0]:
def lstm_model_predict(path, look_back=7):

  dataframe = read_csv(path, usecols=[1], engine='python')

  dataset = dataframe.values
  dataset = dataset.astype('float32')

  scaler = MinMaxScaler(feature_range=(0, 1))
  dataset = scaler.fit_transform(dataset)

  X, Y = create_dataset(dataset, look_back)
  X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

  model = Sequential()
  model.add(LSTM(2, input_shape=(1, look_back)))
  model.add(Dropout(0.2))
  model.add(Dense(2))
  model.add(Dropout(0.2))
  model.add(Dense(2))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')

  split = int(len(X)*0.8)
  model.fit(X[split:], Y[split:], epochs =1000, batch_size=32,validation_data=(X[:split], Y[:split]),verbose=0)
  precision = 1 - model.evaluate(X, Y)
  
  init = 0
  next_days = []
  x = np.array([X[-1]], dtype=np.float32)

  while init < 31: # 31 días siguientes
    predict = model.predict(x)
    x[0][0] = [x[0][0][i] for i in range(1, len(x[0][0]))] + [predict[0][0]]
    next_days.append( int(np.floor(scaler.inverse_transform(predict)[0][0])) )
    init += 1

  print(precision)
  plt.plot( np.append( scaler.inverse_transform(dataset), next_days) )
  plt.show()
  
  return precision, next_days

In [0]:
path = "/content/drive/My Drive/Colab Notebooks/pronóstico de bolsas/data-proc-total.csv"
precision, next_days = lstm_model_predict(path, look_back=100)
data = ["Total diario", precision] + next_days

ValueError: ignored

In [0]:
path = "/content/drive/My Drive/Colab Notebooks/pronóstico de bolsas/output-total.csv"
np.savetxt(path, data, delimiter=",", fmt='%s')